In [1]:
import pyvisa
from qcodes.instrument_drivers.rohde_schwarz import (
    RohdeSchwarzZNB8,
    RohdeSchwarzZNBChannel,
)
import matplotlib.pyplot as plt
import matplotlib
%matplotlib qt

import numpy as np
from numpy import *
from scipy.fft import fft, ifft, fftshift
from scipy.io import savemat
from datetime import datetime
import time

import os
Global_data_adress = r"C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA"
exp_name = 'TWPA_noise'
adress = Global_data_adress+'\\'+exp_name
if not os.path.exists(adress):
  os.mkdir(adress)
  print("Folder %s created!" % adress)
else:
  print("Folder %s already exists" % adress)

Folder C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise already exists


Create a folder for measurement data

In [2]:
now = datetime.now()
path = adress +'\\'+ now.strftime('%Y-%m-%d')
if not os.path.exists(path):
  os.mkdir(path)
  print("Folder %s created!" % path)
else:
  print("Folder %s already exists" % path)

Folder C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise\2023-09-20 created!


# VNA init

In [3]:
def magtodb(mag): return 20*np.log10(mag)

def vna_setup_CW(cw_frequency=6e9, vna_power=-30, bandwidth=10):
    try: print(vna.query('*IDN?'))                 # check if connection already exist
    except:                                        # if no connection
        rm = pyvisa.ResourceManager()
        vna = rm.open_resource('GPIB0::20::INSTR') # GPIB connection
        print(vna.query('*IDN?'))
        
    vna.write(':SENS1:BAND {}'.format(str(bandwidth))) # set defoult bandwidth
    vna.write('SOUR1:POW {}'.format(str(vna_power)))   # set defoult power
    vna.write('INIT1:CONT OFF')                        # single-sweep mode
    vna.write('SENS1:SWE:POIN 201')                    # set maximum possible number of points for single sweep
    vna.write(':SENS1:SWE:TYPE CW')                    # set CW mode of measurements
    return vna


def vna_on(): vna.write('OUTP ON')
def vna_off(): vna.write('OUTP OFF')
    
def vna_meas_points(): 
    return int(vna.query('SENS1:SWE:POIN?'))  # should be 201

def vna_sweep_time(): 
    return float(vna.query('SENS1:SWE:TIME?'))  # estimated sweep time for current settings of vna

def vna_set_cw_freq(freq): 
    vna.write('SENS1:FREQ:CW {}'.format(str(freq))) # set single frequency point for CW mode in Hz

def vna_set_power(power): 
    vna.write('SOUR1:POW {}'.format(str(power))) # set power
    
def vna_set_band(bandwidth): 
    vna.write(':SENS1:BAND {}'.format(str(bandwidth))) # set power
    
def vna_meas_cw():
    vna_on()
    vna.write("INIT1:IMM")                              # start measurement
    time.sleep(0.2+float(vna.query('SENS1:SWE:TIME?'))) # this part is important becouse the measurement nedds to finnish before readout process
    vna.write("CALC1:FORM MLOG")                        # format of data
    data_str = vna.query("CALC1:DATA? SDAT")            # readout
    
    data = np.array(data_str.rstrip().split(",")).astype("float64")
    s = data[0::2] + 1j * data[1::2]
    i = data[0::2]
    q = data[1::2]
    return abs(s), angle(s)

In [4]:
vna = vna_setup_CW() # initializing or checking for connection

Rohde-Schwarz,ZNB20-2Port,1311601062102129,3.45



# Single time sweep

# ANAPICO init

In [5]:
def setup_anapico(power=-30, frequency = 6e9, channel = 1):
    try: print(anapico.query('*IDN?'))
    except: 
        rm = pyvisa.ResourceManager()
    #     anapico = rm.open_resource('USB0::0x03EB::0xAFFF::3C6-0B4F40003-0985::INSTR') # USB connection
        anapico = rm.open_resource('TCPIP0::169.254.5.91::inst0::INSTR') # LAN connection
        print(anapico.query('*IDN?'))
    if power<=-6:
#         print('Flux power set:', power)
        anapico.write(r'SOUR{}:POW {}'.format(str(channel), str(power)))
        anapico.write(r'SOUR{}:FREQ {}'.format(str(channel), str(frequency)))
    else: 
        print('Flux power is too high:', power)
    return anapico

def anapico_on():
    anapico.write(r'OUTP1 ON')

def anapico_off():
    anapico.write(r'OUTP1 OFF')

def anapico_freq(freq, channel=1): 
    anapico.write(r'SOUR{}:FREQ {}'.format(str(channel), str(freq)))

def anapico_power(power, channel=1): 
    anapico.write(r'SOUR{}:POW {}'.format(str(channel), str(power)))

In [6]:
anapico = setup_anapico()

AnaPico AG,APMS20G,3C6-0B4F40003-0985,0.4.204



In [10]:
anapico.close()

In [89]:
# Check time domain data
vna_set_cw_freq(6e9)
vna_set_power(-20)
anapico_power(-20)

mag, pha = vna_meas_cw()
print('number of points measured {} of {}'.format(len(pha),int(vna_meas_points())))

fig, ax = plt.subplots(1,2,figsize=(15,7), sharex=True)
ax[0].plot(fft_f(), mag)
# ax[0].plot(fft_f(), magtodb(mag))
ax[1].plot(fft_f(), pha)
ax[0].set_xlabel('Frequency, GHz')
ax[1].set_xlabel('Frequency, GHz')
ax[0].set_ylabel('Magnitude')
ax[1].set_ylabel('Phase')

number of points measured 201 of 201


NameError: name 'fft_f' is not defined

# Spectrum for single set of parameters

In [83]:
#_____________ANAPICO______________
anapico_power(-15)
flux_freq = 11.252e9 # maximum gain frequency
anapico_freq(flux_freq)
anapico_on()


#_____________VNA______________
nop = 30
vna_freqs = np.linspace(5.63e9, 5.633e9, nop)  #resonance frequency
vna_bandwidth = 10
number_of_points = vna_meas_points()
vna_power = -10

avg_total = 4  #separate measurements of vna per each flux power

In [84]:
vna_set_power(vna_power)
vna_set_band(vna_bandwidth)

mag_s = np.array([])
pha_s = np.array([])

mag_s_err = np.array([])
pha_s_err = np.array([])

now1 = datetime.now()
print('Start: {}'.format(now.strftime(r'%H-%M-%S')))
for en_f, vna_freq in enumerate(vna_freqs):
    print('round {}\{}, current flux freq = {}'.format(en_f, len(vna_freqs), round(vna_freq/1e9,2)))
    vna_set_cw_freq(vna_freq)
    
    try: mag, pha = vna_meas_cw()
    except: 
        vna = vna_setup_CW(cw_frequency=vna_freq, vna_power=vna_power, bandwidth=vna_bandwidth)
        mag, pha = vna_meas_cw()
    mag_s = np.append(mag_s, mean(mag))
    pha_s = np.append(pha_s, mean(pha))
    
    mag_s_err = np.append(mag_s_err, mag.std())
    pha_s_err = np.append(pha_s_err, pha.std())

fig, ax = plt.subplots(1,2,figsize=(15,7), sharex=True)
ax[0].errorbar(vna_freqs, mag_s, xerr=0, yerr=mag_s_err, label = 'Max gain')
ax[1].errorbar(vna_freqs, pha_s, xerr=0, yerr=pha_s_err, label = 'Max gain')
now2 = datetime.now()
print('Run time:', now2-now1)

round 0\30, current flux freq = 5.63
round 1\30, current flux freq = 5.63
round 2\30, current flux freq = 5.63
round 3\30, current flux freq = 5.63
round 4\30, current flux freq = 5.63
round 5\30, current flux freq = 5.63
round 6\30, current flux freq = 5.63
round 7\30, current flux freq = 5.63
round 8\30, current flux freq = 5.63
round 9\30, current flux freq = 5.63
round 10\30, current flux freq = 5.63
round 11\30, current flux freq = 5.63
round 12\30, current flux freq = 5.63
round 13\30, current flux freq = 5.63
round 14\30, current flux freq = 5.63
round 15\30, current flux freq = 5.63
round 16\30, current flux freq = 5.63
round 17\30, current flux freq = 5.63
round 18\30, current flux freq = 5.63
round 19\30, current flux freq = 5.63
round 20\30, current flux freq = 5.63
round 21\30, current flux freq = 5.63
round 22\30, current flux freq = 5.63
round 23\30, current flux freq = 5.63
round 24\30, current flux freq = 5.63
round 25\30, current flux freq = 5.63
round 26\30, current f

<ErrorbarContainer object of 3 artists>

In [87]:
fig, ax = plt.subplots(1,2,figsize=(15,7), sharex=True)
ax[0].set_xlabel('Frequency, GHz')
ax[1].set_xlabel('Frequency, GHz')
ax[0].set_ylabel('Std(Magnitude)')
ax[1].set_ylabel('Std(Phase)')
ax[0].errorbar(vna_freqs/1e9, mag_s_err, label = 'Max gain')
ax[1].errorbar(vna_freqs/1e9, pha_s_err, label = 'Max gain')
ax[0].legend()
ax[1].legend()
fig.savefig(path+'\\'+'Spectrum TWPA'+'_-10VNA_'+now.strftime(r'%H-%M-%S')+'.png', dpi = 200, transparent = False)

In [88]:
dict_data=dict()
dict_data['flux_powers'] = -15
dict_data['flux_freqs'] = flux_freq
dict_data['vna_freqs'] = vna_freqs
dict_data['vna_bandwidth'] = vna_bandwidth
dict_data['number_of_points'] = number_of_points
dict_data['vna_power'] = vna_power
dict_data['avg_total'] = avg_total

dict_data['mag_s'] = mag_s
dict_data['pha_s'] = pha_s

dict_data['mag_s_err'] = mag_s_err
dict_data['pha_s_err'] = pha_s_err

experiment_name = 'Spectrum TWPA'
file_name = path+'\\'+experiment_name+'_'+now.strftime(r'%H-%M-%S')+'.mat'
savemat(file_name, dict_data)

Create log file

In [41]:
comment = 'TWPA spectrum, standard deviation'


file_name = adress +'\\'+'log_'+exp_name+'.txt'
f = open(file_name, 'a')  # append  to existing file
f.write(str(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))+':   ')
f.write(comment+'\n')
f.close()

# Spectrum power sweep

In [8]:
#_____________ANAPICO______________
flux_powers = np.linspace(-25, -6, 5)
flux_freqs = np.linspace(11.252e9-2000e6, 11.252e9+2000e6, 10) # 11.252e9 maximum gain frequency

#_____________VNA______________
nop = 15
vna_freqs = np.linspace(5.629e9, 5.633e9, nop)  #resonance frequency
vna_bandwidth = 10
number_of_points = vna_meas_points()
vna_powers = np.linspace(-25, 0, 5)


In [9]:
vna_set_power(-10)
vna_set_band(vna_bandwidth)

data_to_save =   np.empty((len(flux_freqs), len(vna_powers), len(vna_freqs), 2))
data_to_save_err=np.empty((len(flux_freqs), len(vna_powers), len(vna_freqs), 2))


for en_fp, flux_power in enumerate(flux_powers):
    fig, ax = plt.subplots(1,2,figsize=(15,7), sharex=True)
    
    for en_ff, flux_freq in enumerate(flux_freqs):
        for en_vp, vna_power in enumerate(vna_powers):
            for en_vf, vna_freq in enumerate(vna_freqs):  
                
                anapico_power(flux_power)
                anapico_freq(flux_freq)
                vna_set_power(vna_power)
                vna_set_cw_freq(vna_freq)

                print('round [{}\{}].[{}\{}].[{}\{}].[{}\{}]'.format(en_fp, len(flux_powers), en_ff, len(flux_freqs),
                                                                      en_vp, len(vna_powers), en_vf, len(vna_freqs)))
                      
                try: mag, pha = vna_meas_cw()
                except: 
                    vna = vna_setup_CW(cw_frequency=vna_freq, vna_power=vna_power, bandwidth=vna_bandwidth)
                    mag, pha = vna_meas_cw()
                      
                data_to_save[en_ff, en_vp, en_vf ,0] = mean(mag)
                data_to_save[en_ff, en_vp, en_vf ,1] = mean(pha)

                data_to_save_err[en_ff, en_vp, en_vf ,0] = mag.std()
                data_to_save_err[en_ff, en_vp, en_vf ,1] = pha.std()
                
    for en_vp, vna_power in enumerate(vna_powers):
        ax[0].plot(vna_freqs, data_to_save_err[0, en_vp, : ,0],
                       label = 'VNA_Pow = {}'.format(round(vna_power,2)))

        ax[1].plot(vna_freqs, data_to_save_err[0, en_vp, : ,1], 
                       label = 'VNA_Pow = {}'.format(round(vna_power,2)))
    
                
    dict_data=dict()
    dict_data['flux_power'] = flux_power
    dict_data['flux_freqs'] = flux_freqs
    dict_data['vna_powers'] = vna_powers
    dict_data['vna_freqs'] = vna_freqs
    
    dict_data['vna_bandwidth'] = vna_bandwidth
    dict_data['number_of_points'] = number_of_points
#     dict_data['avg_total'] = avg_total

    dict_data['data_to_save'] = data_to_save
    dict_data['data_to_save_err'] = data_to_save_err
    
    experiment_name = 'FluxPow_'+str(flux_power)
    file_name = path+'\\'+experiment_name+'_'+now.strftime(r'%H-%M-%S')+'.mat'
    savemat(file_name, dict_data)

round [0\5].[0\10].[0\5].[0\15]
round [0\5].[0\10].[0\5].[1\15]
round [0\5].[0\10].[0\5].[2\15]
round [0\5].[0\10].[0\5].[3\15]
round [0\5].[0\10].[0\5].[4\15]
round [0\5].[0\10].[0\5].[5\15]
round [0\5].[0\10].[0\5].[6\15]
round [0\5].[0\10].[0\5].[7\15]
round [0\5].[0\10].[0\5].[8\15]
round [0\5].[0\10].[0\5].[9\15]
round [0\5].[0\10].[0\5].[10\15]
round [0\5].[0\10].[0\5].[11\15]
round [0\5].[0\10].[0\5].[12\15]
round [0\5].[0\10].[0\5].[13\15]
round [0\5].[0\10].[0\5].[14\15]
round [0\5].[0\10].[1\5].[0\15]
round [0\5].[0\10].[1\5].[1\15]
round [0\5].[0\10].[1\5].[2\15]
round [0\5].[0\10].[1\5].[3\15]
round [0\5].[0\10].[1\5].[4\15]
round [0\5].[0\10].[1\5].[5\15]
round [0\5].[0\10].[1\5].[6\15]
round [0\5].[0\10].[1\5].[7\15]
round [0\5].[0\10].[1\5].[8\15]
round [0\5].[0\10].[1\5].[9\15]
round [0\5].[0\10].[1\5].[10\15]
round [0\5].[0\10].[1\5].[11\15]
round [0\5].[0\10].[1\5].[12\15]
round [0\5].[0\10].[1\5].[13\15]
round [0\5].[0\10].[1\5].[14\15]
round [0\5].[0\10].[2\5].[0\15

round [0\5].[3\10].[1\5].[14\15]
round [0\5].[3\10].[2\5].[0\15]
round [0\5].[3\10].[2\5].[1\15]
round [0\5].[3\10].[2\5].[2\15]
round [0\5].[3\10].[2\5].[3\15]
round [0\5].[3\10].[2\5].[4\15]
round [0\5].[3\10].[2\5].[5\15]
round [0\5].[3\10].[2\5].[6\15]
round [0\5].[3\10].[2\5].[7\15]
round [0\5].[3\10].[2\5].[8\15]
round [0\5].[3\10].[2\5].[9\15]
round [0\5].[3\10].[2\5].[10\15]
round [0\5].[3\10].[2\5].[11\15]
round [0\5].[3\10].[2\5].[12\15]
round [0\5].[3\10].[2\5].[13\15]
round [0\5].[3\10].[2\5].[14\15]
round [0\5].[3\10].[3\5].[0\15]
round [0\5].[3\10].[3\5].[1\15]
round [0\5].[3\10].[3\5].[2\15]
round [0\5].[3\10].[3\5].[3\15]
round [0\5].[3\10].[3\5].[4\15]
round [0\5].[3\10].[3\5].[5\15]
round [0\5].[3\10].[3\5].[6\15]
round [0\5].[3\10].[3\5].[7\15]
round [0\5].[3\10].[3\5].[8\15]
round [0\5].[3\10].[3\5].[9\15]
round [0\5].[3\10].[3\5].[10\15]
round [0\5].[3\10].[3\5].[11\15]
round [0\5].[3\10].[3\5].[12\15]
round [0\5].[3\10].[3\5].[13\15]
round [0\5].[3\10].[3\5].[14\1

round [0\5].[6\10].[3\5].[13\15]
round [0\5].[6\10].[3\5].[14\15]
round [0\5].[6\10].[4\5].[0\15]
round [0\5].[6\10].[4\5].[1\15]
round [0\5].[6\10].[4\5].[2\15]
round [0\5].[6\10].[4\5].[3\15]
round [0\5].[6\10].[4\5].[4\15]
round [0\5].[6\10].[4\5].[5\15]
round [0\5].[6\10].[4\5].[6\15]
round [0\5].[6\10].[4\5].[7\15]
round [0\5].[6\10].[4\5].[8\15]
round [0\5].[6\10].[4\5].[9\15]
round [0\5].[6\10].[4\5].[10\15]
round [0\5].[6\10].[4\5].[11\15]
round [0\5].[6\10].[4\5].[12\15]
round [0\5].[6\10].[4\5].[13\15]
round [0\5].[6\10].[4\5].[14\15]
round [0\5].[7\10].[0\5].[0\15]
round [0\5].[7\10].[0\5].[1\15]
round [0\5].[7\10].[0\5].[2\15]
round [0\5].[7\10].[0\5].[3\15]
round [0\5].[7\10].[0\5].[4\15]
round [0\5].[7\10].[0\5].[5\15]
round [0\5].[7\10].[0\5].[6\15]
round [0\5].[7\10].[0\5].[7\15]
round [0\5].[7\10].[0\5].[8\15]
round [0\5].[7\10].[0\5].[9\15]
round [0\5].[7\10].[0\5].[10\15]
round [0\5].[7\10].[0\5].[11\15]
round [0\5].[7\10].[0\5].[12\15]
round [0\5].[7\10].[0\5].[13\1

round [1\5].[0\10].[0\5].[12\15]
round [1\5].[0\10].[0\5].[13\15]
round [1\5].[0\10].[0\5].[14\15]
round [1\5].[0\10].[1\5].[0\15]
round [1\5].[0\10].[1\5].[1\15]
round [1\5].[0\10].[1\5].[2\15]
round [1\5].[0\10].[1\5].[3\15]
round [1\5].[0\10].[1\5].[4\15]
round [1\5].[0\10].[1\5].[5\15]
round [1\5].[0\10].[1\5].[6\15]
round [1\5].[0\10].[1\5].[7\15]
round [1\5].[0\10].[1\5].[8\15]
round [1\5].[0\10].[1\5].[9\15]
round [1\5].[0\10].[1\5].[10\15]
round [1\5].[0\10].[1\5].[11\15]
round [1\5].[0\10].[1\5].[12\15]
round [1\5].[0\10].[1\5].[13\15]
round [1\5].[0\10].[1\5].[14\15]
round [1\5].[0\10].[2\5].[0\15]
round [1\5].[0\10].[2\5].[1\15]
round [1\5].[0\10].[2\5].[2\15]
round [1\5].[0\10].[2\5].[3\15]
round [1\5].[0\10].[2\5].[4\15]
round [1\5].[0\10].[2\5].[5\15]
round [1\5].[0\10].[2\5].[6\15]
round [1\5].[0\10].[2\5].[7\15]
round [1\5].[0\10].[2\5].[8\15]
round [1\5].[0\10].[2\5].[9\15]
round [1\5].[0\10].[2\5].[10\15]
round [1\5].[0\10].[2\5].[11\15]
round [1\5].[0\10].[2\5].[12\1

round [1\5].[3\10].[2\5].[11\15]
round [1\5].[3\10].[2\5].[12\15]
round [1\5].[3\10].[2\5].[13\15]
round [1\5].[3\10].[2\5].[14\15]
round [1\5].[3\10].[3\5].[0\15]
round [1\5].[3\10].[3\5].[1\15]
round [1\5].[3\10].[3\5].[2\15]
round [1\5].[3\10].[3\5].[3\15]
round [1\5].[3\10].[3\5].[4\15]
round [1\5].[3\10].[3\5].[5\15]
round [1\5].[3\10].[3\5].[6\15]
round [1\5].[3\10].[3\5].[7\15]
round [1\5].[3\10].[3\5].[8\15]
round [1\5].[3\10].[3\5].[9\15]
round [1\5].[3\10].[3\5].[10\15]
round [1\5].[3\10].[3\5].[11\15]
round [1\5].[3\10].[3\5].[12\15]
round [1\5].[3\10].[3\5].[13\15]
round [1\5].[3\10].[3\5].[14\15]
round [1\5].[3\10].[4\5].[0\15]
round [1\5].[3\10].[4\5].[1\15]
round [1\5].[3\10].[4\5].[2\15]
round [1\5].[3\10].[4\5].[3\15]
round [1\5].[3\10].[4\5].[4\15]
round [1\5].[3\10].[4\5].[5\15]
round [1\5].[3\10].[4\5].[6\15]
round [1\5].[3\10].[4\5].[7\15]
round [1\5].[3\10].[4\5].[8\15]
round [1\5].[3\10].[4\5].[9\15]
round [1\5].[3\10].[4\5].[10\15]
round [1\5].[3\10].[4\5].[11\1

...

round [3\5].[3\10].[4\5].[5\15]
round [3\5].[3\10].[4\5].[6\15]
round [3\5].[3\10].[4\5].[7\15]
round [3\5].[3\10].[4\5].[8\15]
round [3\5].[3\10].[4\5].[9\15]
round [3\5].[3\10].[4\5].[10\15]
round [3\5].[3\10].[4\5].[11\15]
round [3\5].[3\10].[4\5].[12\15]
round [3\5].[3\10].[4\5].[13\15]
round [3\5].[3\10].[4\5].[14\15]
round [3\5].[4\10].[0\5].[0\15]
round [3\5].[4\10].[0\5].[1\15]
round [3\5].[4\10].[0\5].[2\15]
round [3\5].[4\10].[0\5].[3\15]
round [3\5].[4\10].[0\5].[4\15]
round [3\5].[4\10].[0\5].[5\15]
round [3\5].[4\10].[0\5].[6\15]
round [3\5].[4\10].[0\5].[7\15]
round [3\5].[4\10].[0\5].[8\15]
round [3\5].[4\10].[0\5].[9\15]
round [3\5].[4\10].[0\5].[10\15]
round [3\5].[4\10].[0\5].[11\15]
round [3\5].[4\10].[0\5].[12\15]
round [3\5].[4\10].[0\5].[13\15]
round [3\5].[4\10].[0\5].[14\15]
round [3\5].[4\10].[1\5].[0\15]
round [3\5].[4\10].[1\5].[1\15]
round [3\5].[4\10].[1\5].[2\15]
round [3\5].[4\10].[1\5].[3\15]
round [3\5].[4\10].[1\5].[4\15]
round [3\5].[4\10].[1\5].[5\15

round [3\5].[7\10].[1\5].[4\15]
round [3\5].[7\10].[1\5].[5\15]
round [3\5].[7\10].[1\5].[6\15]
round [3\5].[7\10].[1\5].[7\15]
round [3\5].[7\10].[1\5].[8\15]
round [3\5].[7\10].[1\5].[9\15]
round [3\5].[7\10].[1\5].[10\15]
round [3\5].[7\10].[1\5].[11\15]
round [3\5].[7\10].[1\5].[12\15]
round [3\5].[7\10].[1\5].[13\15]
round [3\5].[7\10].[1\5].[14\15]
round [3\5].[7\10].[2\5].[0\15]
round [3\5].[7\10].[2\5].[1\15]
round [3\5].[7\10].[2\5].[2\15]
round [3\5].[7\10].[2\5].[3\15]
round [3\5].[7\10].[2\5].[4\15]
round [3\5].[7\10].[2\5].[5\15]
round [3\5].[7\10].[2\5].[6\15]
round [3\5].[7\10].[2\5].[7\15]
round [3\5].[7\10].[2\5].[8\15]
round [3\5].[7\10].[2\5].[9\15]
round [3\5].[7\10].[2\5].[10\15]
round [3\5].[7\10].[2\5].[11\15]
round [3\5].[7\10].[2\5].[12\15]
round [3\5].[7\10].[2\5].[13\15]
round [3\5].[7\10].[2\5].[14\15]
round [3\5].[7\10].[3\5].[0\15]
round [3\5].[7\10].[3\5].[1\15]
round [3\5].[7\10].[3\5].[2\15]
round [3\5].[7\10].[3\5].[3\15]
round [3\5].[7\10].[3\5].[4\15

round [4\5].[0\10].[3\5].[3\15]
round [4\5].[0\10].[3\5].[4\15]
round [4\5].[0\10].[3\5].[5\15]
round [4\5].[0\10].[3\5].[6\15]
round [4\5].[0\10].[3\5].[7\15]
round [4\5].[0\10].[3\5].[8\15]
round [4\5].[0\10].[3\5].[9\15]
round [4\5].[0\10].[3\5].[10\15]
round [4\5].[0\10].[3\5].[11\15]
round [4\5].[0\10].[3\5].[12\15]
round [4\5].[0\10].[3\5].[13\15]
round [4\5].[0\10].[3\5].[14\15]
round [4\5].[0\10].[4\5].[0\15]
round [4\5].[0\10].[4\5].[1\15]
round [4\5].[0\10].[4\5].[2\15]
round [4\5].[0\10].[4\5].[3\15]
round [4\5].[0\10].[4\5].[4\15]
round [4\5].[0\10].[4\5].[5\15]
round [4\5].[0\10].[4\5].[6\15]
round [4\5].[0\10].[4\5].[7\15]
round [4\5].[0\10].[4\5].[8\15]
round [4\5].[0\10].[4\5].[9\15]
round [4\5].[0\10].[4\5].[10\15]
round [4\5].[0\10].[4\5].[11\15]
round [4\5].[0\10].[4\5].[12\15]
round [4\5].[0\10].[4\5].[13\15]
round [4\5].[0\10].[4\5].[14\15]
round [4\5].[1\10].[0\5].[0\15]
round [4\5].[1\10].[0\5].[1\15]
round [4\5].[1\10].[0\5].[2\15]
round [4\5].[1\10].[0\5].[3\15

round [4\5].[4\10].[0\5].[2\15]
round [4\5].[4\10].[0\5].[3\15]
round [4\5].[4\10].[0\5].[4\15]
round [4\5].[4\10].[0\5].[5\15]
round [4\5].[4\10].[0\5].[6\15]
round [4\5].[4\10].[0\5].[7\15]
round [4\5].[4\10].[0\5].[8\15]
round [4\5].[4\10].[0\5].[9\15]
round [4\5].[4\10].[0\5].[10\15]
round [4\5].[4\10].[0\5].[11\15]
round [4\5].[4\10].[0\5].[12\15]
round [4\5].[4\10].[0\5].[13\15]
round [4\5].[4\10].[0\5].[14\15]
round [4\5].[4\10].[1\5].[0\15]
round [4\5].[4\10].[1\5].[1\15]
round [4\5].[4\10].[1\5].[2\15]
round [4\5].[4\10].[1\5].[3\15]
round [4\5].[4\10].[1\5].[4\15]
round [4\5].[4\10].[1\5].[5\15]
round [4\5].[4\10].[1\5].[6\15]
round [4\5].[4\10].[1\5].[7\15]
round [4\5].[4\10].[1\5].[8\15]
round [4\5].[4\10].[1\5].[9\15]
round [4\5].[4\10].[1\5].[10\15]
round [4\5].[4\10].[1\5].[11\15]
round [4\5].[4\10].[1\5].[12\15]
round [4\5].[4\10].[1\5].[13\15]
round [4\5].[4\10].[1\5].[14\15]
round [4\5].[4\10].[2\5].[0\15]
round [4\5].[4\10].[2\5].[1\15]
round [4\5].[4\10].[2\5].[2\15

round [4\5].[7\10].[2\5].[1\15]
round [4\5].[7\10].[2\5].[2\15]
round [4\5].[7\10].[2\5].[3\15]
round [4\5].[7\10].[2\5].[4\15]
round [4\5].[7\10].[2\5].[5\15]
round [4\5].[7\10].[2\5].[6\15]
round [4\5].[7\10].[2\5].[7\15]
round [4\5].[7\10].[2\5].[8\15]
round [4\5].[7\10].[2\5].[9\15]
round [4\5].[7\10].[2\5].[10\15]
round [4\5].[7\10].[2\5].[11\15]
round [4\5].[7\10].[2\5].[12\15]
round [4\5].[7\10].[2\5].[13\15]
round [4\5].[7\10].[2\5].[14\15]
round [4\5].[7\10].[3\5].[0\15]
round [4\5].[7\10].[3\5].[1\15]
round [4\5].[7\10].[3\5].[2\15]
round [4\5].[7\10].[3\5].[3\15]
round [4\5].[7\10].[3\5].[4\15]
round [4\5].[7\10].[3\5].[5\15]
round [4\5].[7\10].[3\5].[6\15]
round [4\5].[7\10].[3\5].[7\15]
round [4\5].[7\10].[3\5].[8\15]
round [4\5].[7\10].[3\5].[9\15]
round [4\5].[7\10].[3\5].[10\15]
round [4\5].[7\10].[3\5].[11\15]
round [4\5].[7\10].[3\5].[12\15]
round [4\5].[7\10].[3\5].[13\15]
round [4\5].[7\10].[3\5].[14\15]
round [4\5].[7\10].[4\5].[0\15]
round [4\5].[7\10].[4\5].[1\15

In [20]:
fig, ax = plt.subplots(1,2,figsize=(15,7), sharex=True)

en_fp = 1
en_ff = 1 

st = fig.suptitle("TWPA power {}, TWPA freq {}".format(round(flux_powers[en_fp],2), round(flux_freqs[en_ff]/1e9,5)), fontsize="x-large")

# set_xlim(0)
# ax[0].set_ylim(-30, 0)
# ax[1].set_ylim(-30, 0)
for en_vp, vna_power in enumerate(vna_powers):

    ax[0].plot(vna_freqs, data_to_save_err[en_fp, en_ff, en_vp, : ,0],
                   label = 'VNA_Pow = {}'.format(round(vna_power,2)))

    ax[1].plot(vna_freqs, data_to_save_err[en_fp, en_ff, en_vp, : ,1], 
                   label = 'VNA_Pow = {}'.format(round(vna_power,2)))
    
#     ax[0].errorbar(vna_freqs, data_to_save[en_fp, en_ff, en_vp, : ,0], 
#                    xerr=0, yerr=data_to_save_err[en_fp, en_ff, en_vp, : ,0],
#                    label = 'VNA_Pow = {}'.format(round(vna_power,2)))

#     ax[1].errorbar(vna_freqs, data_to_save[en_fp, en_ff, en_vp, : , 1], 
#                    xerr=0, yerr=data_to_save_err[en_fp, en_ff, en_vp, : ,1], 
#                    label = 'VNA_Pow = {}'.format(round(vna_power,2)))

NameError: name 'data_to_save_err' is not defined

In [78]:
data_to_save_err


array([[[[[ 1.96645940e-004,  1.05768308e-002],
          [ 1.88040376e-004,  1.15758868e-002],
          [ 1.81848444e-004,  1.15567111e-002],
          ...,
          [ 1.85055523e-004,  1.32929592e-002],
          [ 1.82938374e-004,  1.14961855e-002],
          [ 1.72706675e-004,  1.21222334e-002]],

         [[ 8.81323072e-005,  5.72384140e-003],
          [ 7.51305970e-005,  5.48888918e-003],
          [ 9.35661467e-005,  5.94886721e-003],
          ...,
          [ 9.14463502e-005,  6.02771947e-003],
          [ 8.90479211e-005,  5.83288078e-003],
          [ 8.11966713e-005,  5.27029035e-003]],

         [[ 4.72032191e-005,  2.88835170e-003],
          [ 4.33792639e-005,  2.81684872e-003],
          [ 4.67941090e-005,  2.63618646e-003],
          ...,
          [ 4.00590096e-005,  3.14460900e-003],
          [ 4.43449454e-005,  2.93984588e-003],
          [ 4.64556596e-005,  2.97802429e-003]],

         [[ 2.24795570e-005,  1.42611924e-003],
          [ 2.11600108e-005,  1.42365

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15,7), sharex=True)

en_fp = 1
en_vp = 1 

st = fig.suptitle("VNA power {}, TWPA power {}".format(round(vna_power[en_vp],2), round(flux_freqs[en_ff]/1e9,5)), fontsize="x-large")

for en_ff, flux_freq in enumerate(flux_freqs):

    ax[0].errorbar(vna_freqs, data_to_save[en_fp, en_ff, en_vp, : ,0], 
                   xerr=0, yerr=data_to_save_err[en_fp, en_ff, en_vp, : ,0],
                   label = 'Flux_freqs = {}'.format(round(flux_freqs[en_ff]/1e9,5)))

    ax[1].errorbar(vna_freqs, data_to_savedata_to_save[en_fp, en_ff, en_vp, : , 1], 
                   xerr=0, yerr=data_to_save_err[en_fp, en_ff, en_vp, : ,1], 
                   label = 'Flux_freqs = {}'.format(round(flux_freqs[en_ff]/1e9,5)))

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15,7), sharex=True)
ax[0].set_xlabel('Frequency, GHz')
ax[1].set_xlabel('Frequency, GHz')
ax[0].set_ylabel('Std(Magnitude)')
ax[1].set_ylabel('Std(Phase)')
ax[0].errorbar(vna_freqs/1e9, mag_s_err, label = 'Max gain')
ax[1].errorbar(vna_freqs/1e9, pha_s_err, label = 'Max gain')
ax[0].legend()
ax[1].legend()

In [12]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [20]:
a = np.array([0, 5, 17])
print(np.argwhere(a==find_nearest(a, value=20))[0,0])

2
